In [2]:
import boto3
import pandas as pd
from io import StringIO

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
arg_date = '2022-03-15'

In [4]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('xetra-1234')

In [27]:
bucket_obj1 = bucket.objects.filter(Prefix='2022-03-15')
bucket_obj2 = bucket.objects.filter(Prefix='2022-03-16')
objects = [obj for obj in bucket_obj1] + [obj for obj in bucket_obj2]

In [12]:
objects

[s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR00.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR01.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR02.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR03.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR04.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR05.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR06.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR07.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR08.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR09.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR10.csv'),
 s3.Object

In [28]:
csv_obj_init = bucket.Object(key=objects[0].key).get().get('Body').read().decode('utf-8')
data = StringIO(csv_obj_init)
df_init = pd.read_csv(data, delimiter=',')

In [29]:
df_init.columns

Index(['ISIN', 'Mnemonic', 'SecurityDesc', 'SecurityType', 'Currency',
       'SecurityID', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume', 'NumberOfTrades'],
      dtype='object')

In [38]:
df_all = pd.DataFrame(columns=df_init.columns)
for obj in objects:
    csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    df_all = pd.concat([df_all, df], ignore_index = True)


In [20]:
df_all

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
0,DE000A0DJ6J9,S92,SMA SOLAR TECHNOL.AG,Common stock,EUR,2504287,2022-03-15,08:00,37.34,37.76,37.30,37.50,11421,35
1,DE000A0D6554,NDX1,NORDEX SE O.N.,Common stock,EUR,2504290,2022-03-15,08:00,16.58,16.58,16.43,16.45,21537,37
2,DE000A0D9PT0,MTX,MTU AERO ENGINES NA O.N.,Common stock,EUR,2504297,2022-03-15,08:00,201.10,202.40,201.00,202.40,1585,29
3,DE000A0JL9W6,VBK,VERBIO VER.BIOENERGIE ON,Common stock,EUR,2504343,2022-03-15,08:00,62.55,63.20,62.55,63.20,1539,8
4,DE000A0LD6E6,GXI,GERRESHEIMER AG,Common stock,EUR,2504380,2022-03-15,08:00,61.40,61.40,60.95,61.05,329,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251797,US02005N1000,GMZ,"ALLY FINANCIAL INC.DL-,10",Common stock,EUR,5988595,2022-03-16,20:30,37.80,37.80,37.80,37.80,0,1
251798,US2533931026,DSG,"DICK'S SPORTING DL-,01",Common stock,EUR,5990122,2022-03-16,20:30,101.00,101.00,101.00,101.00,0,1
251799,US89832Q1094,BBK,TRUIST FINL CORP. DL 5,Common stock,EUR,6001418,2022-03-16,20:30,52.50,52.50,52.50,52.50,0,1
251800,GB00BNYK8G86,7TJ,"DARKTRACE PLC LS-,01",Common stock,EUR,7026059,2022-03-16,20:30,5.20,5.20,5.20,5.20,0,1


In [39]:
columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
df_all = df_all.loc[:, columns]

In [32]:
df_all.head(10)

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume
0,DE000A0DJ6J9,2022-03-15,08:00,37.34,37.76,37.30,37.50,11421
1,DE000A0D6554,2022-03-15,08:00,16.58,16.58,16.43,16.45,21537
2,DE000A0D9PT0,2022-03-15,08:00,201.10,202.40,201.00,202.40,1585
3,DE000A0JL9W6,2022-03-15,08:00,62.55,63.20,62.55,63.20,1539
4,DE000A0LD6E6,2022-03-15,08:00,61.40,61.40,60.95,61.05,329
5,DE000A0WMPJ6,2022-03-15,08:00,17.60,17.60,17.60,17.60,3451
6,DE000A0Z2ZZ5,2022-03-15,08:00,24.00,24.03,23.95,24.03,10167
7,DE000A1DAHH0,2022-03-15,08:00,69.98,70.12,69.94,70.02,3238
8,DE000A1EWWW0,2022-03-15,08:00,205.00,205.50,203.00,203.35,21880
9,DE000A1H8BV3,2022-03-15,08:00,27.76,27.78,27.60,27.64,1064


In [40]:
df_all.dropna(inplace=True)
df_all.shape

(251802, 8)

## Get opening perice per ISIN and day

In [41]:
df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('first')

In [18]:
df_all[df_all['ISIN']=='AT000000STR1']

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price
30657,AT000000STR1,2022-03-15,09:45,36.40,36.40,36.40,36.40,9,36.4
34007,AT000000STR1,2022-03-15,09:59,36.65,36.65,36.65,36.65,107,36.4
39613,AT000000STR1,2022-03-15,10:27,37.00,37.00,37.00,37.00,120,36.4
54772,AT000000STR1,2022-03-15,11:44,37.55,37.55,37.55,37.55,300,36.4
122790,AT000000STR1,2022-03-15,16:35,37.05,37.05,37.05,37.05,0,36.4
127988,AT000000STR1,2022-03-16,08:05,37.50,37.50,37.50,37.50,50,37.5
128531,AT000000STR1,2022-03-16,08:07,37.50,37.50,37.50,37.50,450,37.5
174326,AT000000STR1,2022-03-16,11:22,37.55,37.55,37.55,37.55,50,37.5
186559,AT000000STR1,2022-03-16,12:17,37.55,37.55,37.55,37.55,200,37.5
242845,AT000000STR1,2022-03-16,16:11,37.40,37.40,37.40,37.40,6,37.5


## Get closing perice per ISIN and day

In [42]:
df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('last')

In [36]:
df_all[df_all['ISIN']=='AT000000STR1']

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price,closing_price
30657,AT000000STR1,2022-03-15,09:45,36.40,36.40,36.40,36.40,9,36.4,37.05
34007,AT000000STR1,2022-03-15,09:59,36.65,36.65,36.65,36.65,107,36.4,37.05
39613,AT000000STR1,2022-03-15,10:27,37.00,37.00,37.00,37.00,120,36.4,37.05
54772,AT000000STR1,2022-03-15,11:44,37.55,37.55,37.55,37.55,300,36.4,37.05
122790,AT000000STR1,2022-03-15,16:35,37.05,37.05,37.05,37.05,0,36.4,37.05
127988,AT000000STR1,2022-03-16,08:05,37.50,37.50,37.50,37.50,50,37.5,37.15
128531,AT000000STR1,2022-03-16,08:07,37.50,37.50,37.50,37.50,450,37.5,37.15
174326,AT000000STR1,2022-03-16,11:22,37.55,37.55,37.55,37.55,50,37.5,37.15
186559,AT000000STR1,2022-03-16,12:17,37.55,37.55,37.55,37.55,200,37.5,37.15
242845,AT000000STR1,2022-03-16,16:11,37.40,37.40,37.40,37.40,6,37.5,37.15


## Aggregations

In [43]:
df_all = df_all.groupby(['ISIN','Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), 
                                             minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'),
                                            daily_raded_volume=('TradedVolume', 'sum'))

In [44]:
df_all.head(20)

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_raded_volume
0,AT000000STR1,2022-03-15,36.40,37.05,36.40,37.55,536
1,AT000000STR1,2022-03-16,37.50,37.15,37.15,37.55,816
2,AT00000FACC2,2022-03-15,7.88,7.71,7.61,7.88,68
3,AT00000FACC2,2022-03-16,7.95,8.07,7.95,8.08,696
4,AT0000606306,2022-03-15,13.30,13.36,12.88,13.57,24002
5,AT0000606306,2022-03-16,13.90,14.52,13.74,15.12,55655
6,AT0000609607,2022-03-15,11.38,11.14,11.14,11.38,14
7,AT0000609607,2022-03-16,11.64,11.80,11.64,11.80,5
8,AT0000644505,2022-03-15,92.30,91.30,91.30,92.40,241
9,AT0000644505,2022-03-16,93.70,96.30,93.70,96.30,217


## Percent Change Prev Closing

In [45]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

In [46]:
df_all.head(20)

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_raded_volume,prev_closing_price
0,AT000000STR1,2022-03-15,36.40,37.05,36.40,37.55,536,NaN
1,AT000000STR1,2022-03-16,37.50,37.15,37.15,37.55,816,37.05
2,AT00000FACC2,2022-03-15,7.88,7.71,7.61,7.88,68,NaN
3,AT00000FACC2,2022-03-16,7.95,8.07,7.95,8.08,696,7.71
4,AT0000606306,2022-03-15,13.30,13.36,12.88,13.57,24002,NaN
5,AT0000606306,2022-03-16,13.90,14.52,13.74,15.12,55655,13.36
6,AT0000609607,2022-03-15,11.38,11.14,11.14,11.38,14,NaN
7,AT0000609607,2022-03-16,11.64,11.80,11.64,11.80,5,11.14
8,AT0000644505,2022-03-15,92.30,91.30,91.30,92.40,241,NaN
9,AT0000644505,2022-03-16,93.70,96.30,93.70,96.30,217,91.30


In [48]:
df_all['change_prev_closing_%'] = df_all['closing_price_eur'] - df_all['prev_closing_price'] / df_all['prev_closing_price'] * 100

In [49]:
df_all.head(10)

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_raded_volume,prev_closing_price,change_prev_closing_%
0,AT000000STR1,2022-03-15,36.40,37.05,36.40,37.55,536,NaN,NaN
1,AT000000STR1,2022-03-16,37.50,37.15,37.15,37.55,816,37.05,-62.85
2,AT00000FACC2,2022-03-15,7.88,7.71,7.61,7.88,68,NaN,NaN
3,AT00000FACC2,2022-03-16,7.95,8.07,7.95,8.08,696,7.71,-91.93
4,AT0000606306,2022-03-15,13.30,13.36,12.88,13.57,24002,NaN,NaN
5,AT0000606306,2022-03-16,13.90,14.52,13.74,15.12,55655,13.36,-85.48
6,AT0000609607,2022-03-15,11.38,11.14,11.14,11.38,14,NaN,NaN
7,AT0000609607,2022-03-16,11.64,11.80,11.64,11.80,5,11.14,-88.20
8,AT0000644505,2022-03-15,92.30,91.30,91.30,92.40,241,NaN,NaN
9,AT0000644505,2022-03-16,93.70,96.30,93.70,96.30,217,91.30,-3.70


In [50]:
df_all.drop(columns=['prev_closing_price'], inplace=True)

In [51]:
df_all = df_all.round(decimals=2)